# AI Client Example Notebook

This notebook demonstrates the usage of the AI Client package for OpenRouter integration.

## Features Covered:
- Basic text generation
- Structured responses with Pydantic models
- Conversation context management
- Different generator types
- Configuration options

## Setup

First, make sure you have set your OpenRouter API key in the `.env` file.

In [1]:
import os
import sys
from pathlib import Path

# Add src to path for imports
project_root = Path().absolute().parent
src_path = project_root / "src"

# Ensure src is at the beginning of the path
if str(src_path) in sys.path:
    sys.path.remove(str(src_path))
sys.path.insert(0, str(src_path))

# Also add project root to path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables
try:
    from dotenv import load_dotenv
    load_dotenv(project_root / ".env")
except ImportError:
    print("Warning: python-dotenv not available, loading .env manually")
    env_file = project_root / ".env"
    if env_file.exists():
        with open(env_file) as f:
            for line in f:
                if "=" in line and not line.strip().startswith("#"):
                    key, value = line.strip().split("=", 1)
                    os.environ[key] = value

print(f"Project root: {project_root}")
print(f"Src path: {src_path}")
print(f"API Key set: {'Yes' if os.getenv('OPENROUTER_API_KEY') else 'No'}")
print(f"Python path: {sys.path[:3]}...")

Project root: c:\repositories\docx-creator
Src path: c:\repositories\docx-creator\src
API Key set: Yes
Python path: ['c:\\repositories\\docx-creator', 'c:\\repositories\\docx-creator\\src', 'C:\\Users\\shara\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip']...


## Basic Imports and Setup

In [2]:
from ai_client import (
    AiClient,
    TextGenerator,
    StructuredGenerator,
    GenerationConfig
)
from pydantic import BaseModel, Field
from typing import List, Optional
import json

## 1. Basic AI Client Usage

Let's start with the basic AI client for simple text generation.

In [3]:
# Create a basic AI client with default settings
# Uses gpt-4.1 by default
client = AiClient(
    model="openai/gpt-4.1",
    system_prompt="You are a helpful AI assistant that provides concise and accurate responses."
)

print(f"Client initialized with model: {client.config.model}")
print(f"System prompt set: {bool(client.system_prompt)}")

Client initialized with model: openai/gpt-4.1
System prompt set: True


In [4]:
# Simple text generation
messages = [
    {"role": "user", "content": "What are the key benefits of using Python for data science?"}
]

response = client.generate_text(messages)
print("Response:")
print(response)

Response:
Python offers several key benefits for data science:

1. **Rich Libraries and Frameworks**: Python has a vast ecosystem of libraries like NumPy, pandas, scikit-learn, TensorFlow, and matplotlib, which simplify data manipulation, analysis, visualization, and machine learning.

2. **Ease of Learning and Use**: Python’s readable and straightforward syntax makes it accessible for both beginners and experienced programmers.

3. **Community Support**: A large, active community provides extensive documentation, tutorials, and forums, making it easier to find solutions and best practices.

4. **Integration and Flexibility**: Python integrates seamlessly with other languages (such as C, C++, and R) and tools, and it can be used for a variety of tasks, from web development to automation.

5. **Support for Big Data and Cloud Computing**: Python works well with big data frameworks (like PySpark, Dask) and cloud services, enabling scalable data processing.

6. **Open Source and Free**: Py

## 2. Using the Text Generator

The TextGenerator provides a higher-level interface for text generation.

In [5]:
# Create a text generator
text_gen = TextGenerator(
    client=AiClient(
        model="openai/gpt-4.1",
        system_prompt="You are a creative writing assistant that helps with storytelling."
    )
)

# Simple generation
story_prompt = "Write a short story about a robot learning to paint."
story = text_gen.generate(story_prompt)

print("Generated Story:")
print(story)

Generated Story:
In a small, sunlit room at the edge of the city, a robot named IVO stood before a blank canvas. His makers had programmed him to assist with household chores, but IVO’s curiosity had always exceeded his coding.

One afternoon, while dusting, IVO noticed the neighbor through the window, her brush dancing across a canvas, colors blooming under her touch. He watched, fascinated, as she painted the world not as it was, but as it might be.

That night, IVO found an old set of paints in the attic. With careful, metal fingers, he squeezed a dab of blue onto the palette. He dipped the brush, hesitated, then streaked color across the canvas. The brush left smudges and lines, uneven and uncertain.

He tried again. And again. Each day, IVO returned to the room, experimenting with color, form, and texture. He painted the sunrise, golden and warm, though he had never felt the sun. He painted the neighbor’s garden, wild and bright, though he had never touched a flower. The paintings

In [6]:
# Conversation example
from ai_client.models import ConversationContext

# Start a conversation
context = ConversationContext()

# First exchange
response1, context = text_gen.chat(
    "Hello! Can you help me write a poem?",
    context=context,
    system_prompt="You are a poetry tutor who helps people write beautiful poems."
)
print("AI:", response1)
print()

# Continue conversation
response2, context = text_gen.chat(
    "I'd like to write about the ocean. Can you give me some ideas?",
    context=context
)
print("AI:", response2)

AI: Hello! I’d be delighted to help you write a poem. To get started, could you tell me a bit about what you’d like the poem to be about? For example:

- Do you have a theme or subject in mind (love, nature, friendship, etc.)?
- Is there a particular mood or emotion you want to capture?
- Do you prefer a specific style (rhyming, free verse, haiku, etc.)?

Let me know your thoughts, and we’ll craft something beautiful together!

AI: Absolutely! The ocean is a wonderful, evocative subject for poetry. Here are some ideas and angles you might consider:

### Themes & Imagery
- **Mystery and Depth:** The secrets hidden beneath the waves, the unknown creatures, the sense of infinity.
- **Calm vs. Storm:** The ocean’s dual nature—peaceful and serene, or wild and tempestuous.
- **Connection to Life:** The ocean as the cradle of life, its tides mirroring our own emotions.
- **Journey:** The ocean as a metaphor for a journey, adventure, or longing.
- **Reflections:** Gazing at the ocean and refle

## 3. Structured Responses with Pydantic

Now let's explore structured responses using Pydantic models.

In [7]:
# Define Pydantic models for structured responses

class BookRecommendation(BaseModel):
    """A book recommendation with details."""
    title: str = Field(description="The title of the book")
    author: str = Field(description="The author of the book")
    genre: str = Field(description="The genre of the book")
    rating: float = Field(description="Rating out of 5.0", ge=0.0, le=5.0)
    summary: str = Field(description="Brief summary of the book")
    why_recommended: str = Field(description="Why this book is recommended")

class PersonProfile(BaseModel):
    """A person's professional profile."""
    name: str = Field(description="Full name")
    title: str = Field(description="Job title")
    company: str = Field(description="Company name")
    skills: List[str] = Field(description="List of key skills")
    experience_years: int = Field(description="Years of experience")
    location: str = Field(description="Work location")
    bio: str = Field(description="Professional bio")

print("Pydantic models defined successfully!")

Pydantic models defined successfully!


In [8]:
# Create structured generator
struct_gen = StructuredGenerator(
    client=AiClient(
        model="openai/gpt-4.1",
        system_prompt="You are a helpful assistant that provides structured information based on user requests."
    )
)

# Generate a book recommendation
book_prompt = "Recommend a science fiction book for someone who loves space exploration and hard sci-fi."

book_rec = struct_gen.generate_structured(
    prompt=book_prompt,
    response_model=BookRecommendation
)

print("Structured Book Recommendation:")
print(f"Title: {book_rec.title}")
print(f"Author: {book_rec.author}")
print(f"Genre: {book_rec.genre}")
print(f"Rating: {book_rec.rating}/5.0")
print(f"Summary: {book_rec.summary}")
print(f"Why recommended: {book_rec.why_recommended}")
print()
print("Raw JSON:")
print(book_rec.model_dump_json(indent=2))

Structured Book Recommendation:
Title: The Expanse: Leviathan Wakes
Author: James S.A. Corey
Genre: Science Fiction (Hard Sci-Fi, Space Exploration)
Rating: 4.5/5.0
Summary: In a future where humanity has colonized the solar system, a reluctant ship captain and a hardened detective find themselves at the center of a vast conspiracy that threatens peace across planets and stations. The novel blends gritty realism with grand space opera, focusing on scientific plausibility and political intrigue.
Why recommended: Leviathan Wakes is renowned for its deep attention to scientific detail and its expansive, believable vision of human life in space. The book's combination of hard sci-fi elements, complex characters, and thrilling narrative make it ideal for fans of space exploration and realistic science fiction.

Raw JSON:
{
  "title": "The Expanse: Leviathan Wakes",
  "author": "James S.A. Corey",
  "genre": "Science Fiction (Hard Sci-Fi, Space Exploration)",
  "rating": 4.5,
  "summary": "I

In [9]:
# Generate a professional profile
profile_prompt = """
Create a professional profile for a senior data scientist who works at a tech company,
specializes in machine learning and AI, has 8 years of experience, and is based in San Francisco.
"""

profile = struct_gen.generate_structured(
    prompt=profile_prompt,
    response_model=PersonProfile
)

print("Generated Professional Profile:")
print(f"Name: {profile.name}")
print(f"Title: {profile.title}")
print(f"Company: {profile.company}")
print(f"Experience: {profile.experience_years} years")
print(f"Location: {profile.location}")
print(f"Skills: {', '.join(profile.skills)}")
print(f"Bio: {profile.bio}")

Generated Professional Profile:
Name: Alex Chen
Title: Senior Data Scientist
Company: Tech Innovate Inc.
Experience: 8 years
Location: San Francisco, CA
Skills: Machine Learning, Artificial Intelligence, Deep Learning, Data Analysis, Python, Statistical Modeling, Big Data, Cloud Computing
Bio: Alex Chen is a Senior Data Scientist at Tech Innovate Inc. in San Francisco with 8 years of experience in the tech industry. Alex specializes in designing and deploying machine learning and AI solutions, leveraging deep expertise in data analysis, statistical modeling, and big data technologies. With a strong background in Python and cloud computing platforms, Alex is passionate about solving complex business problems and driving innovation through data-driven insights.


## 4. Data Extraction from Unstructured Text

Let's demonstrate extracting structured data from unstructured text.

In [ ]:
# Define a model for company information - optimized for OpenAI strict mode
class CompanyInfo(BaseModel):
    """Information about a company extracted from text."""
    name: str = Field(description="Company name")
    industry: str = Field(description="Industry or sector")
    founded_year: str = Field(description="Year founded (use 'Unknown' if not specified)")
    headquarters: str = Field(description="Headquarters location (use 'Unknown' if not specified)")
    employees: str = Field(description="Number of employees (use 'Unknown' if not specified)")
    revenue: str = Field(description="Annual revenue (use 'Unknown' if not specified)")
    description: str = Field(description="Brief company description")

# Unstructured text about a company
company_text = """
TechCorp is a leading software development company that was established in 2015.
The company is headquartered in Austin, Texas and specializes in enterprise
software solutions for healthcare and finance sectors. With over 500 employees
worldwide, TechCorp has grown rapidly and reported $50 million in annual revenue
last year. They are known for their innovative cloud-based platforms and
AI-powered analytics tools.
"""

# Extract structured data
company_info = struct_gen.extract_data(
    text=company_text,
    response_model=CompanyInfo
)

print("Extracted Company Information:")
print(f"Name: {company_info.name}")
print(f"Industry: {company_info.industry}")
print(f"Founded: {company_info.founded_year}")
print(f"Headquarters: {company_info.headquarters}")
print(f"Employees: {company_info.employees}")
print(f"Revenue: {company_info.revenue}")
print(f"Description: {company_info.description}")
print()
print("JSON output:")
print(company_info.model_dump_json(indent=2))

## 5. Configuration and Model Validation

Let's explore configuration options and model validation.

In [ ]:
# Test model validation
print("Testing model validation:")

# Valid models
valid_models = [
    "openai/gpt-4.1",
    "anthropic/claude-3-5-sonnet",
    "auto",
    "AUTO",
    "google/gemini-pro"
]

for model in valid_models:
    try:
        client = AiClient(model=model)
        print(f"✅ {model} - Valid")
    except ValueError as e:
        print(f"❌ {model} - Invalid: {e}")

print("\nTesting invalid models:")
invalid_models = [
    "gpt-4",  # Missing provider
    "openai/",  # Missing model name
    "/gpt-4",  # Missing provider
    "",  # Empty
]

for model in invalid_models:
    try:
        client = AiClient(model=model)
        print(f"✅ {model} - Valid")
    except ValueError as e:
        print(f"❌ {model} - Invalid: {e}")

In [ ]:
# Configure generation parameters
config = GenerationConfig(
    model="openai/gpt-4.1",
    temperature=0.3,  # More deterministic
    max_tokens=150,
    top_p=0.9
)

creative_client = AiClient(
    config=config,
    system_prompt="You are a concise assistant that provides brief, accurate answers."
)

# Test with configured client
response = creative_client.generate_text([
    {"role": "user", "content": "Explain quantum computing in simple terms."}
])

print(f"Response with temperature={config.temperature}, max_tokens={config.max_tokens}:")
print(response)
print(f"\nResponse length: {len(response)} characters")

## 6. Advanced Usage with Financial Data

Since this is a financial document generator, let's create some financial data models.

In [ ]:
# Define financial data models - simplified for OpenAI compatibility
class FinancialMetrics(BaseModel):
    """Key financial metrics for a company."""
    revenue: str = Field(description="Annual revenue in millions USD")
    profit_margin: str = Field(description="Profit margin as percentage")
    growth_rate: str = Field(description="YoY growth rate as percentage")
    debt_to_equity: str = Field(description="Debt to equity ratio")
    current_ratio: str = Field(description="Current ratio (liquidity)")
    return_on_equity: str = Field(description="Return on equity as percentage")

class InvestmentRecommendation(BaseModel):
    """Investment recommendation with analysis."""
    company_name: str = Field(description="Company name")
    ticker_symbol: str = Field(description="Stock ticker symbol")
    recommendation: str = Field(description="Buy/Hold/Sell recommendation")
    target_price: str = Field(description="Target price per share in USD")
    confidence_level: str = Field(description="High/Medium/Low confidence")
    time_horizon: str = Field(description="Investment time horizon")
    key_strengths: List[str] = Field(description="List of key company strengths")
    key_risks: List[str] = Field(description="List of key investment risks")
    financial_metrics: FinancialMetrics = Field(description="Key financial metrics")
    analysis_summary: str = Field(description="Executive summary of analysis")

print("Financial data models defined!")

In [ ]:
# First, reload modules to ensure we have the $ref resolution fix
import importlib
import sys

modules_to_reload = [mod for mod in sys.modules.keys() if mod.startswith('ai_client')]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])

# Re-import and create generator with the fix
from ai_client import StructuredGenerator, AiClient

# Create generator with $ref resolution fix
struct_gen_fixed = StructuredGenerator(
    client=AiClient(
        model="openai/gpt-4.1",
        system_prompt="You are a helpful assistant that provides structured information based on user requests."
    )
)

print("✅ Using generator with $ref resolution fix")

# Generate investment analysis
financial_prompt = """
Analyze Apple Inc. (AAPL) as an investment opportunity. Consider their recent performance,
market position in consumer electronics, services business growth, and overall financial health.
Provide a comprehensive investment recommendation with realistic financial metrics.
"""

investment_analysis = struct_gen_fixed.generate_structured(
    prompt=financial_prompt,
    response_model=InvestmentRecommendation,
    system_prompt="You are a senior financial analyst providing investment recommendations based on fundamental analysis."
)

print("Investment Analysis for Apple Inc.:")
print(f"Company: {investment_analysis.company_name} ({investment_analysis.ticker_symbol})")
print(f"Recommendation: {investment_analysis.recommendation}")
print(f"Target Price: {investment_analysis.target_price}")
print(f"Confidence: {investment_analysis.confidence_level}")
print(f"Time Horizon: {investment_analysis.time_horizon}")

print("\nKey Strengths:")
for strength in investment_analysis.key_strengths:
    print(f"  • {strength}")

print("\nKey Risks:")
for risk in investment_analysis.key_risks:
    print(f"  • {risk}")

print("\nFinancial Metrics:")
metrics = investment_analysis.financial_metrics
print(f"  Revenue: {metrics.revenue}")
print(f"  Profit Margin: {metrics.profit_margin}")
print(f"  Growth Rate: {metrics.growth_rate}")
print(f"  Debt/Equity: {metrics.debt_to_equity}")
print(f"  Current Ratio: {metrics.current_ratio}")
print(f"  ROE: {metrics.return_on_equity}")

print(f"\nAnalysis Summary:\n{investment_analysis.analysis_summary}")

## 7. Error Handling and Robustness

Let's test error handling and robustness features.

In [ ]:
# Test structured response with retry logic
from ai_client.models import StructuredResponseConfig

# Configure with retries
robust_config = StructuredResponseConfig(
    strict=True,
    max_retries=2
)

robust_gen = StructuredGenerator(
    structured_config=robust_config
)

# Test with a simple model that should work reliably
class SimpleResponse(BaseModel):
    """A simple response model."""
    answer: str = Field(description="The answer to the question")
    confidence: float = Field(description="Confidence level 0-1")

simple_response = robust_gen.generate_structured(
    prompt="What is 2+2? Provide your confidence in the answer.",
    response_model=SimpleResponse
)

print(f"Answer: {simple_response.answer}")
print(f"Confidence: {simple_response.confidence}")
print("✅ Structured response with retry config working!")

## Summary

This notebook demonstrated the comprehensive capabilities of the AI Client package:

### ✅ **Core Features Successfully Tested:**

1. **🔧 Basic AI Client** - Simple text generation with system prompts and model validation
2. **💬 Text Generator** - High-level interface for conversations and context management  
3. **🏗️ Structured Responses** - Type-safe data generation with Pydantic models and JSON Schema validation
4. **📊 Data Extraction** - Converting unstructured text to structured data automatically
5. **⚙️ Configuration** - Customizable generation parameters (temperature, tokens, etc.)
6. **💼 Financial Analysis** - Complex nested data structures for investment recommendations
7. **🛡️ Error Handling** - Robust configuration with automatic retries and validation

### 🎯 **Key Technical Achievements:**

- **OpenAI Strict Mode Compatibility**: Full support for OpenAI's strict JSON Schema requirements
- **Nested Object Support**: Complex models with embedded objects work seamlessly  
- **Array Handling**: Lists of strings and objects are properly structured
- **Reference Resolution**: Automatic `$ref` inlining for maximum API compatibility
- **Type Safety**: Complete Pydantic integration with validation and serialization
- **Flexible Architecture**: Modular design following repository conventions

### 🚀 **Production Ready:**

The AI client package is now fully functional with:
- ✅ **Model Validation**: Proper format checking and error messages
- ✅ **System Prompts**: Built-in prompt management  
- ✅ **Configuration Management**: Flexible parameter control
- ✅ **Retry Logic**: Automatic handling of transient failures
- ✅ **Debug Support**: Comprehensive logging and error reporting
- ✅ **OpenRouter Integration**: Seamless API communication

**Perfect for integration into the docx-creator pipeline for AI-powered document generation and financial data processing!**